In [25]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
#Constants
eu2 = (2/3)**2
ed2 = (-1/3)**2
es2 = (-1/3)**2
alpha = 1/137
hc_factor = 3.89 * 10**8
factor = ((4*np.pi*alpha)**2)/(9*2*np.pi)

In [3]:
NUM_QT_VALUES = 20
NUM_xF_VALUES = 20
BATCH_SIZE = 1024

In [7]:
# Load Data
E288_200 = pd.read_csv("/home/ishara/Documents/TMDs/Ansatz_Approach_for_QM/Work/Data_Updated/E288_200.csv")
E288_300 = pd.read_csv("/home/ishara/Documents/TMDs/Ansatz_Approach_for_QM/Work/Data_Updated/E288_300.csv")
E288_400 = pd.read_csv("/home/ishara/Documents/TMDs/Ansatz_Approach_for_QM/Work/Data_Updated/E288_400.csv")

data = pd.concat([E288_200, E288_300, E288_400], ignore_index=True)

In [8]:
data

,qT,A,PreFactor,CS,dA,error,sysNorm,sysP2P,y,y_Low,...,QM_Low,QM_High,SqrtS,xF,xF_Low,xF_High,x1,x2,BeamE,frac_error
0,0.1,6.290000,1.590909,3.953714,0.320000,0.201143,0.00%,0.00%,0.40,0.361736,...,4,5,19.37008,0.190850,0.169644,0.212055,0.346576,0.155727,200,0.050874
1,0.3,5.880000,0.530303,11.088000,0.240000,0.452571,0.00%,0.00%,0.40,0.361736,...,4,5,19.37008,0.190850,0.169644,0.212055,0.346576,0.155727,200,0.040816
2,0.5,4.750000,0.318182,14.928571,0.230000,0.722857,0.00%,0.00%,0.40,0.361736,...,4,5,19.37008,0.190850,0.169644,0.212055,0.346576,0.155727,200,0.048421
3,0.7,3.690000,0.227273,16.236000,0.220000,0.968000,0.00%,0.00%,0.40,0.361736,...,4,5,19.37008,0.190850,0.169644,0.212055,0.346576,0.155727,200,0.059621
4,0.9,2.930000,0.176768,16.575429,0.220000,1.244571,0.00%,0.00%,0.40,0.361736,...,4,5,19.37008,0.190850,0.169644,0.212055,0.346576,0.155727,200,0.075085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,1.5,0.000190,0.106061,0.001791,0.000080,0.000754,0.00%,0.00%,0.03,0.028929,...,13,14,27.39343,0.029574,0.028478,0.030669,0.507827,0.478254,400,0.421053
380,1.7,0.000090,0.093583,0.000962,0.000050,0.000534,0.00%,0.00%,0.03,0.028929,...,13,14,27.39343,0.029574,0.028478,0.030669,0.507827,0.478254,400,0.555556
381,1.9,0.000110,0.083732,0.001314,0.000070,0.000836,0.00%,0.00%,0.03,0.028929,...,13,14,27.39343,0.029574,0.028478,0.030669,0.507827,0.478254,400,0.636364
382,2.3,0.000057,0.069170,0.000824,0.000041,0.000593,0.00%,0.00%,0.03,0.028929,...,13,14,27.39343,0.029574,0.028478,0.030669,0.507827,0.478254,400,0.719298


In [9]:
x2_unique = np.sort(data['x2'].unique())
x2_value = x2_unique[0] if len(x2_unique) > 0 else x2_min
print(f"Using fixed x2 value: {x2_value}")

Using fixed x2 value: 0.153756551


In [18]:
qT_array = np.array(np.linspace(data['qT'].min(),data['qT'].max(),NUM_QT_VALUES))
xF_array = np.array(np.linspace(data['xF'].min(),data['xF'].max(),NUM_xF_VALUES))
x1_array = xF_array + x2_value
x2_array = np.array(np.linspace(x2_value,x2_value,NUM_xF_VALUES))
SqT_input = np.hstack([qT_array, x1_array, x2_array])

In [19]:
SqT_input

array([0.1       , 0.35263158, 0.60526316, 0.85789474, 1.11052632,
       1.36315789, 1.61578947, 1.86842105, 2.12105263, 2.37368421,
       2.62631579, 2.87894737, 3.13157895, 3.38421053, 3.63684211,
       3.88947368, 4.14210526, 4.39473684, 4.64736842, 4.9       ,
       0.16580504, 0.18860857, 0.21141211, 0.23421564, 0.25701917,
       0.2798227 , 0.30262623, 0.32542977, 0.3482333 , 0.37103683,
       0.39384036, 0.4166439 , 0.43944743, 0.46225096, 0.48505449,
       0.50785802, 0.53066156, 0.55346509, 0.57626862, 0.59907215,
       0.15375655, 0.15375655, 0.15375655, 0.15375655, 0.15375655,
       0.15375655, 0.15375655, 0.15375655, 0.15375655, 0.15375655,
       0.15375655, 0.15375655, 0.15375655, 0.15375655, 0.15375655,
       0.15375655, 0.15375655, 0.15375655, 0.15375655, 0.15375655])

In [22]:
models_folder = 'Models'
if not os.path.exists(models_folder):
    models_folder = '../../Step_by_step_tuning_to_get_sqT/Test_68/Models'

def custom_weighted_loss(y_true, y_pred, w=None):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    if w is not None:
        w = tf.cast(w, tf.float32)
        mean_w = tf.reduce_mean(w)
        weights = w / mean_w
        squared_error = tf.square(y_pred - y_true)
        weighted_squared_error = squared_error * weights
        return tf.reduce_mean(weighted_squared_error)
    # For model loading (when w not available), fall back to MSE
    else:
        return tf.reduce_mean(tf.square(y_pred - y_true))

# Create a wrapper class to make the loss function serializable
class CustomWeightedLoss(tf.keras.losses.Loss):
    def __init__(self, name="custom_weighted_loss"):
        super().__init__(name=name)
    
    def __call__(self, y_true, y_pred, sample_weight=None):
        return custom_weighted_loss(y_true, y_pred)

# Check if models folder exists
if not os.path.exists(models_folder):
    print(f"Error: Models folder '{models_folder}' does not exist.")
    # Create a dummy model for testing if no models are available
    print("Creating a dummy model for testing purposes...")
    input_layer = tf.keras.layers.Input(shape=(3,))
    hidden = tf.keras.layers.Dense(10, activation='relu')(input_layer)
    output = tf.keras.layers.Dense(1)(hidden)
    dummy_model = tf.keras.models.Model(inputs=input_layer, outputs=output, name="dummy_model")
    # Add a layer with name 'SqT' for testing
    SqT_input = tf.keras.layers.Input(shape=(3,))
    hidden = tf.keras.layers.Dense(10, activation='relu')(SqT_input)
    SqT_output = tf.keras.layers.Dense(1)(hidden)
    SqT_model = tf.keras.models.Model(inputs=SqT_input, outputs=SqT_output, name="SqT")
    models_list = [dummy_model]
    # Add SqT as a layer to the dummy model (workaround for testing)
    dummy_model.layers.append(SqT_model)
    dummy_model._layer_by_name = {layer.name: layer for layer in dummy_model.layers}
else:
    # Load All Trained Models with proper custom objects
    model_files = [f for f in os.listdir(models_folder) if f.endswith('.h5')]
    if not model_files:
        print(f"No model files found in '{models_folder}'.")
        exit(1)
        
    models_list = []
    for f in model_files:
        try:
            model = tf.keras.models.load_model(
                os.path.join(models_folder, f), 
                custom_objects={
                    'custom_weighted_loss': custom_weighted_loss,
                    'CustomWeightedLoss': CustomWeightedLoss,
                    'train_weighted_loss': custom_weighted_loss
                }
            )
            models_list.append(model)
        except Exception as e:
            print(f"Error loading model {f}: {e}")

print(f"Loaded {len(models_list)} models.")

I0000 00:00:1746474673.675156   17786 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6360 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5


Loaded 50 models.


In [24]:
for i in range(0, 20):    
    SqT_model_preds = []
    
    for model in models_list:
        SqT_model = model.get_layer('SqT')
        SqT_pred = SqT_model.predict(SqT_input, verbose=0)
        SqT_model_preds.append(SqT_pred)
    
    SqT_all_preds = np.array(SqT_model_preds)
    SqT_mean_batch = np.mean(SqT_all_preds, axis=0).flatten()
    SqT_std_batch = np.std(SqT_all_preds, axis=0).flatten()
    

ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32,), dtype=float32). Expected shape (None, 3), but input has incompatible shape (32,)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(32,), dtype=float32)
  • training=False
  • mask=None